In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, f1_score, precision_score, recall_score
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split

SAVE_PATH = ""  # Data saving folder
PREPROCESS = True  # If False, skip quantum processing and load data from SAVE_PATH
np.random.seed(0)  # Seed for NumPy random number generator
tf.random.set_seed(0)
dataset_root = ""
n_layers = 1  # Number of random layers

def load_custom_dataset(root_folder, image_size=(100, 100)):
    images = []
    labels = []
    class_names = sorted(os.listdir(dataset_root))
    class_map = {class_name: idx for idx, class_name in enumerate(class_names)}
    print(class_names)
    for class_name in class_names:
        class_folder = os.path.join(dataset_root, class_name)
        if os.path.isdir(class_folder):
            for image_name in os.listdir(class_folder):
                image_path = os.path.join(class_folder, image_name)
                try:
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    image = cv2.resize(image, image_size)
                    images.append(image)
                    labels.append(class_map[class_name])
                except Exception as e:
                    print(f"Error loading image: {image_path}, Error: {e}")
    return np.array(images), np.array(labels)

images, labels = load_custom_dataset(dataset_root)
print(f"Loaded {len(images)} images and {len(set(labels))} classes.")
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")
n_train = X_train.shape[0]    # Size of the train dataset
n_test = X_test.shape[0]     # Size of the test dataset
train_images = X_train
test_images = X_test
train_images = train_images / 255
test_images = test_images / 255
train_images = np.array(train_images[..., np.newaxis])
test_images = np.array(test_images[..., np.newaxis])

def create_cnn_model(input_shape, num_classes):
    """Creates and returns a CNN model with backpropagation."""
    model = keras.models.Sequential([
        keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.MaxPooling2D((2, 2)),
        keras.layers.Conv2D(64, (3, 3), activation='relu'),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def compute_metrics(model, test_images, test_labels):
    y_pred = model.predict(test_images)
    predicted_labels = np.argmax(y_pred, axis=1)
    
    # Compute metrics
    precision = precision_score(test_labels, predicted_labels, average='weighted')
    recall = recall_score(test_labels, predicted_labels, average='weighted')
    f1 = f1_score(test_labels, predicted_labels, average='weighted')
    
    return precision, recall, f1

# Define categories (labels) and input shape
categories = ["Class1", "Class2", "Class3"]  # Update if different
input_shape = (50, 50, 4)
num_classes = len(categories)
batch_size = 32
n_epochs = 40

# Load datasets
datasets = {
    'L1_CNOT': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L1_CNOT_01082024_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L1_CNOT_01082024_100.npy"),
    },
    'L2_CNOT': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L2_CNOT_01082024_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L2_CNOT_01082024_100.npy"),
    },
    'L3_CNOT': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L3_CNOT_01082024_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L3_CNOT_01082024_100.npy"),
    },
    'L4_CNOT': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L4_CNOT_01082024_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L4_CNOT_01082024_100.npy"),
    },
    'L5_CNOT': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L5_CNOT_01082024_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L5_CNOT_01082024_100.npy"),
    },
    'L1_CZ': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L1_CZ_01082024_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L1_CZ_01082024_100.npy"),
    },
    'L2_CZ': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L2_CZ_01082024_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L2_CZ_01082024_100.npy"),
    },
    'L3_CZ': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L3_CZ_01082024_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L3_CZ_01082024_100.npy"),
    },
    'L4_CZ': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L4_CZ_01082024_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L4_CZ_01082024_100.npy"),
    },
    'L5_CZ': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L5_CZ_01082024_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L5_CZ_01082024_100.npy"),
    }
}

# Define labels
train_labels = y_train
test_labels = y_test

# Dictionary to store histories and test accuracies
histories = {}
test_accuracies = {}
metrics = {}

# Loop through models, train, and compute metrics
for model_name in datasets.keys():
    # Create and train model
    model = create_cnn_model(input_shape=input_shape, num_classes=num_classes)
    history = model.fit(
        datasets[model_name]['train'], train_labels,
        validation_data=(datasets[model_name]['test'], test_labels),
        epochs=n_epochs,
        batch_size=batch_size,
        verbose=2
    )
    histories[model_name] = history
    test_loss, test_acc = model.evaluate(datasets[model_name]['test'], test_labels, verbose=0)
    test_accuracies[model_name] = test_acc * 100  # Convert to percentage
    
    # Compute additional metrics
    precision, recall, f1 = compute_metrics(model, datasets[model_name]['test'], test_labels)
    metrics[model_name] = {
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Accuracy (%)': test_accuracies[model_name]
    }

# Create a DataFrame to tabulate the metrics
metrics_df = pd.DataFrame(metrics).T  # Transpose for better readability
print(metrics_df)

MNIST Datset CNOT CZ

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset
mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

# Reduce dataset size
n_train, n_test = 60000, 10000  # Modify as needed
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Normalize pixel values
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Add extra dimension for convolution channels
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

# Convert labels to categorical format
num_classes = 10
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

# Define dataset paths
SAVE_PATH = "D:/PhD_FUUAST/mail farina/quanvolution_PC/"

datasets = {
    'M12_CNOT': {
        'train': np.load(SAVE_PATH + "q_train_images_M12.npy"),
        'test': np.load(SAVE_PATH + "q_test_images_M12.npy"),
    },
    'M12_CZ': {
        'train': np.load(SAVE_PATH + "q_train_images_M12_CZ.npy"),
        'test': np.load(SAVE_PATH + "q_test_images_M12_CZ.npy"),
    },
}

# Dictionary to store histories and test accuracies
histories = {}
test_accuracies = {}
metrics = {}

def create_cnn_model(input_shape, num_classes):
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Conv2D(64, (3,3), activation='relu'),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def compute_metrics(model, test_data, test_labels):
    from sklearn.metrics import precision_score, recall_score, f1_score
    y_pred = np.argmax(model.predict(test_data), axis=1)
    y_true = np.argmax(test_labels, axis=1)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return precision, recall, f1

# Model Training Loop
for model_name in datasets.keys():
    print(f"Training on {model_name} dataset...")
    
    # Create and train model
    model = create_cnn_model(input_shape=datasets[model_name]['train'].shape[1:], num_classes=num_classes)
    
    history = model.fit(
        datasets[model_name]['train'], train_labels,
        validation_data=(datasets[model_name]['test'], test_labels),
        epochs=50,  # Adjust as needed
        batch_size=128,
        verbose=2
    )
    
    # Store history
    histories[model_name] = history
    
    # Evaluate on test data
    test_loss, test_acc = model.evaluate(datasets[model_name]['test'], test_labels, verbose=0)
    test_accuracies[model_name] = test_acc * 100  # Convert to percentage
    
    # Compute additional metrics
    precision, recall, f1 = compute_metrics(model, datasets[model_name]['test'], test_labels)
    metrics[model_name] = {
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Accuracy (%)': test_accuracies[model_name]
    }

# Convert to DataFrame for easy visualization
metrics_df = pd.DataFrame(metrics).T
print(metrics_df)


In [ ]:
############################## FRUITQ -SUBSET  ####################################
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import os
import cv2

SAVE_PATH = "______________"  # Data saving folder
PREPROCESS = True  # If False, skip quantum processing and load data from SAVE_PATH
np.random.seed(0)  # Seed for NumPy random number generator
tf.random.set_seed(0)
dataset_root = "__________"
n_layers = 1  # Number of random layers

def load_custom_dataset(root_folder, image_size=(100, 100)):
    images = []
    labels = []
    class_names = sorted(os.listdir(root_folder))
    class_map = {class_name: idx for idx, class_name in enumerate(class_names)}
    print(class_names)

    for class_name in class_names:
        class_folder = os.path.join(root_folder, class_name)
        if os.path.isdir(class_folder):
            for image_name in os.listdir(class_folder):
                image_path = os.path.join(class_folder, image_name)
                try:
                    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                    if image is not None:
                        image = cv2.resize(image, image_size)
                        images.append(image)
                        labels.append(class_map[class_name])
                except Exception as e:
                    print(f"Error loading image: {image_path}, Error: {e}")

    return np.array(images), np.array(labels), len(class_names)

images, labels, num_classes = load_custom_dataset(dataset_root)
print(f"Loaded {len(images)} images and {num_classes} classes.")

# Split data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")

# Normalize images
X_train = X_train / 255.0
X_test = X_test / 255.0

# Reshape for CNN input
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension
X_test = np.expand_dims(X_test, axis=-1)

# Convert labels to categorical format
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Define dataset paths
datasets = {
    'M12_CNOT': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L1_CNOT_FruitQ_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L1_CNOT_FruitQ_100.npy"),
    },
    'M12_CZ': {
        'train': np.load(SAVE_PATH + "2x2_q_train_images_Apple_L1_CZ_FruitQ_100.npy"),
        'test': np.load(SAVE_PATH + "2x2_q_test_images_Apple_L1_CZ_FruitQ_100.npy"),
    },
}

# Dictionary to store histories and test accuracies
histories = {}
test_accuracies = {}
metrics = {}

def create_cnn_model(input_shape, num_classes):
    model = keras.Sequential([
        keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Conv2D(64, (3,3), activation='relu'),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def compute_metrics(model, test_data, test_labels):
    from sklearn.metrics import precision_score, recall_score, f1_score
    y_pred = np.argmax(model.predict(test_data), axis=1)
    y_true = np.argmax(test_labels, axis=1)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    return precision, recall, f1

# Model Training Loop
for model_name in datasets.keys():
    print(f"Training on {model_name} dataset...")

    # Ensure labels match dataset size
    train_data, test_data = datasets[model_name]['train'], datasets[model_name]['test']
    train_labels = y_train[:train_data.shape[0]]
    test_labels = y_test[:test_data.shape[0]]

    # Create and train model
    model = create_cnn_model(input_shape=train_data.shape[1:], num_classes=num_classes)

    history = model.fit(
        train_data, train_labels,
        validation_data=(test_data, test_labels),
        epochs=40,  # Adjust as needed
        batch_size=128,
        verbose=2
    )

    # Store history
    histories[model_name] = history

    # Evaluate on test data
    test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=0)
    test_accuracies[model_name] = test_acc * 100  # Convert to percentage

    # Compute additional metrics
    precision, recall, f1 = compute_metrics(model, test_data, test_labels)
    metrics[model_name] = {
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Accuracy (%)': test_accuracies[model_name]
    }

# Convert to DataFrame for easy visualization
metrics_df = pd.DataFrame(metrics).T
print(metrics_df)
